In [4]:
from functools import wraps


def listify(fn):
    """
    Use this decorator on a generator function to make it return a list
    instead.
    """

    @wraps(fn)
    def listified(*args, **kwargs):
        return list(fn(*args, **kwargs))

    return listified

In [ ]:
def get_featDF(cls, inputDF):
    spark = SparkSession.builder.getOrCreate()
    tmp = cls.add_all_features(inputDF.withColumn("chunkid",
                                                  F.lit("daydate")))
    schema = tmp.schema
    outputDF = spark.createDataFrame([], schema)

    chunk = cls.gen_data_chunks(inputDF, window=cls.window, step=cls.step)
    while True:
        try:
            chunkDF = cls.add_all_features(next(chunk)).cache()
            chunkDF.count()
            outputDF = outputDF.union(chunkDF)
        except StopIteration:
            break

    return outputDF

In [70]:
def rolling(df, schema):
    spark = SparkSession.builder.getOrCreate()
    outdf = spark.createDataFrame([], schema)
    chunkgen = DataLoader.gen_data_chunks(df)
    def rolled(function):
        @wraps
        def wrapper(*args, **kwargs):
            while True:
                try:
                    chunkdf = function(next(chunkgen)).cache()
                    chunkdf.count()
                    outdf = outdf.union(chunkdf)
                except StopIteration:
                    break
            return outdf
        return wrapper
    return rolled

In [ ]:
# replace chunkid with config.TIME_ID_COL
def get_schema(df):
    spark = SparkSession.builder.getOrCreate()
    tmp = cls.add_all_features(df.withColumn("chunkid", F.lit("9999")))
    return tmp.schema